In [1]:
import pandas as pd
import requests

# Function to fetch data from the Census API
def fetch_census_data(url, params):
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data[1:], columns=data[0])
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return pd.DataFrame()

# Define endpoints
url_decennial = 'https://api.census.gov/data/2020/dec/pl'
url_acs = 'https://api.census.gov/data/2020/acs/acs5'

# Parameters for Decennial Census
params_decennial = {
    'get': 'NAME,P1_001N,P1_002N,P1_026N',
    'for': 'county:*',
    'in': 'state:*'
}

# Parameters for ACS (income data)
params_acs = {
    'get': 'NAME,B19013_001E,B19013_001M',
    'for': 'county:*',
    'in': 'state:*'
}

# Fetch data from both APIs
df_decennial = fetch_census_data(url_decennial, params_decennial)
df_acs = fetch_census_data(url_acs, params_acs)

if not df_decennial.empty and not df_acs.empty:
    # Rename columns for clarity
    df_decennial.rename(columns={
        'NAME': 'County_State',
        'P1_001N': 'Total_Population',
        'P1_002N': 'Male_Population',
        'P1_026N': 'Female_Population',
        'state': 'State_Code',
        'county': 'County_Code'
    }, inplace=True)

    df_acs.rename(columns={
        'NAME': 'County_State',
        'B19013_001E': 'Median_Income',
        'B19013_001M': 'Margin_Error_Income',
        'state': 'State_Code',
        'county': 'County_Code'
    }, inplace=True)

    # Convert numeric columns
    for col in ['Total_Population', 'Male_Population', 'Female_Population', 'Median_Income', 'Margin_Error_Income']:
        df_decennial[col] = pd.to_numeric(df_decennial[col], errors='coerce')
        df_acs[col] = pd.to_numeric(df_acs[col], errors='coerce')

    # Merge datasets on common keys
    df = pd.merge(df_decennial, df_acs[['County_State', 'State_Code', 'County_Code', 'Median_Income', 'Margin_Error_Income']],
                  on=['County_State', 'State_Code', 'County_Code'], how='inner')

    # Calculate percentages for male and female population
    df['Percent_Male'] = (df['Male_Population'] / df['Total_Population'] * 100).round(2)
    df['Percent_Female'] = (df['Female_Population'] / df['Total_Population'] * 100).round(2)

    # Map state codes to state names
    state_mapping = {
        '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
        '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'District of Columbia',
        '12': 'Florida', '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois',
        '18': 'Indiana', '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana',
        '23': 'Maine', '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
        '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada',
        '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York',
        '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma', '41': 'Oregon',
        '42': 'Pennsylvania', '44': 'Rhode Island', '45': 'South Carolina', '46': 'South Dakota',
        '47': 'Tennessee', '48': 'Texas', '49': 'Utah', '50': 'Vermont', '51': 'Virginia',
        '53': 'Washington', '54': 'West Virginia', '55': 'Wisconsin', '56': 'Wyoming'
    }
    df['State_Name'] = df['State_Code'].map(state_mapping)

    # Reorder columns for better readability
    df = df[['County_State', 'State_Name', 'County_Code', 'Total_Population', 'Male_Population', 
             'Female_Population', 'Percent_Male', 'Percent_Female', 'Median_Income', 'Margin_Error_Income']]

    # Save the final dataset to a CSV file
    csv_filename = 'census_data_income.csv'
    df.to_csv(csv_filename, index=False)
    print(f"Data successfully saved to {csv_filename}")
else:
    print("Error in fetching data from the APIs.")


KeyError: 'Total_Population'